In [ ]:
import itertools

def parse_expression(expression):
    expression = expression.replace('V', ' and ')
    expression = expression.replace('U', ' or ')
    expression = expression.replace('~', ' not ')

    expression = expression.replace('->', ' or not ')
    import re

    expression = re.sub(r'([A-Z])', r'(\1)', expression)

    return expression

def evaluate_expression(expression, truth_assignment):
    for symbol, value in truth_assignment.items():
        expression = expression.replace(f'({symbol})', str(value))
    try:
        return eval(expression)
    except Exception as e:
        print(f"Error evaluating expression: {expression} with assignment {truth_assignment} - {e}")
        return False


def propositional_inference_enumeration_single(literals, knowledge_base_expr, alpha_expr):
    symbols = sorted(literals)
    truth_assignments = list(itertools.product([True, False], repeat=len(symbols)))
    header = symbols + [knowledge_base_expr, alpha_expr, "KB True", "KB => Alpha"]
    print("\t".join(header))
    print("-" * (8 * len(header)))

    entails = True
    for assignment_values in truth_assignments:
        truth_assignment = dict(zip(symbols, assignment_values))
        parsed_kb_expr = parse_expression(knowledge_base_expr)
        kb_evaluation = evaluate_expression(parsed_kb_expr, truth_assignment)

        parsed_alpha_expr = parse_expression(alpha_expr)
        alpha_evaluation = evaluate_expression(parsed_alpha_expr, truth_assignment)


        kb_true = kb_evaluation
        kb_implies_alpha = not kb_true or alpha_evaluation

        row_values = list(assignment_values) + [kb_evaluation, alpha_evaluation, kb_true, kb_implies_alpha]


        if kb_true and alpha_evaluation:
            print("\t".join(f"\033[1m{v}\033[0m" for v in row_values))
        else:
            print("\t".join(str(v) for v in row_values))


        if kb_true and not alpha_evaluation:
            entails = False

    return entails


num_literals = int(input("Enter the number of literals: "))
literals = [input(f"Enter literal {i+1}: ") for i in range(num_literals)]
knowledge_base_expr = input("Enter the knowledge base expression (use V for AND, U for OR, ~ for NOT, -> for IMPLIES): ")
alpha_expr = input("Enter the alpha expression (use V for AND, U for OR, ~ for NOT, -> for IMPLIES): ")

print("\n--- Truth Table ---")
result = propositional_inference_enumeration_single(literals, knowledge_base_expr, alpha_expr)

if result:
    print(f"\nResult: Knowledge base entails alpha.")
else:
    print(f"\nResult: Knowledge base does not entail alpha.")

Enter the number of literals: 3
Enter literal 1: P
Enter literal 2: M
Enter literal 3: D
Enter the knowledge base expression (use V for AND, U for OR, ~ for NOT, -> for IMPLIES): (~P -> ~M) U ( P -> D)
Enter the alpha expression (use V for AND, U for OR, ~ for NOT, -> for IMPLIES): D -> M

--- Truth Table ---
D	M	P	(~P -> ~M) U ( P -> D)	D -> M	KB True	KB => Alpha
--------------------------------------------------------
True	True	True	True	True	True	True
True	True	False	True	True	True	True
True	False	True	True	True	True	True
True	False	False	True	True	True	True
False	True	True	True	False	True	False
False	True	False	True	False	True	False
False	False	True	True	True	True	True
False	False	False	True	True	True	True

Result: Knowledge base does not entail alpha.


In [ ]:
import itertools
import re

def parse_expression(expression):
    """
    Convert a propositional logic expression (with U, V, ~, ->)
    into a valid Python boolean expression for eval().
    """
    expr = expression.upper().replace(" ", "")

    while "->" in expr:
        expr = re.sub(
            r'\(([^\(\)]+)\)->\(([^\(\)]+)\)',
            r'(not (\1) or (\2))',
            expr
        )
        expr = re.sub(
            r'([A-Z])->\(([^\(\)]+)\)',
            r'(not \1 or (\2))',
            expr
        )
        expr = re.sub(
            r'\(([^\(\)]+)\)->([A-Z])',
            r'(not (\1) or \2)',
            expr
        )
        expr = re.sub(
            r'([A-Z])->([A-Z])',
            r'(not \1 or \2)',
            expr
        )

    expr = expr.replace('U', ' and ')
    expr = expr.replace('V', ' or ')
    expr = expr.replace('~', ' not ')

    return expr


def evaluate_expression(expression, truth_assignment):
    expr = expression
    for symbol, value in truth_assignment.items():
        expr = re.sub(rf'\b{symbol}\b', str(value), expr)
    try:
        return eval(expr)
    except Exception as e:
        print(f"⚠️ Error evaluating expression: {expr} with {truth_assignment} - {e}")
        return False


def propositional_inference_enumeration_single(literals, kb_expr, alpha_expr):
    symbols = sorted(literals)
    truth_assignments = list(itertools.product([True, False], repeat=len(symbols)))

    print("\n" + "\t".join(symbols + ["KB", "Alpha", "KB => Alpha"]))
    print("-" * (8 * (len(symbols) + 3)))

    entails = True
    parsed_kb = parse_expression(kb_expr)
    parsed_alpha = parse_expression(alpha_expr)

    for values in truth_assignments:
        truth_assignment = dict(zip(symbols, values))
        kb_val = evaluate_expression(parsed_kb, truth_assignment)
        alpha_val = evaluate_expression(parsed_alpha, truth_assignment)
        kb_implies_alpha = (not kb_val) or alpha_val

        row = list(map(str, values)) + [str(kb_val), str(alpha_val), str(kb_implies_alpha)]
        if kb_val and not alpha_val:
            print("\033[91m" + "\t".join(row) + "\033[0m")
            entails = False
        else:
            print("\t".join(row))

    return entails


# ---- MAIN ----
if __name__ == "__main__":
    num_literals = int(input("Enter the number of literals: "))
    literals = [input(f"Enter literal {i+1}: ").strip().upper() for i in range(num_literals)]
    kb_expr = input("Enter the knowledge base (use U=AND, V=OR, ~ for NOT, -> for IMPLIES): ").strip()
    alpha_expr = input("Enter the alpha expression (use U=AND, V=OR, ~ for NOT, -> for IMPLIES): ").strip()

    print("\n--- Truth Table ---")
    result = propositional_inference_enumeration_single(literals, kb_expr, alpha_expr)

    print("\nResult:", "✅ Knowledge base entails Alpha" if result else "❌ Knowledge base does not entail Alpha")


Enter the number of literals: 3
Enter literal 1: h
Enter literal 2: m
Enter literal 3: p
Enter the knowledge base (use U=AND, V=OR, ~ for NOT, -> for IMPLIES): (h->m) U (p->h)
Enter the alpha expression (use U=AND, V=OR, ~ for NOT, -> for IMPLIES): ~p V m

--- Truth Table ---

H	M	P	KB	Alpha	KB => Alpha
------------------------------------------------
True	True	True	True	True	True
True	True	False	True	True	True
True	False	True	False	False	True
True	False	False	False	True	True
False	True	True	False	True	True
False	True	False	True	True	True
False	False	True	False	False	True
False	False	False	True	True	True

Result: ✅ Knowledge base entails Alpha
